In [217]:
import yaml
import numpy as np
import pandas as pd
from collections import Counter
from utils import mysqlDatabase
from sklearn.model_selection import train_test_split
with open('config.yml', 'r') as stream:
        myconfig = yaml.load(stream, Loader=yaml.CLoader)
database_username = myconfig['mysql_database']['database_username']
database_password = myconfig['mysql_database']['database_password']
database_ip       = myconfig['mysql_database']['database_ip']
database_name     = myconfig['mysql_database']['database_name']
MysqlDatabase = mysqlDatabase(database_username, database_password, database_ip, database_name)

In [218]:
sql = '''
SELECT df.name, dp.*
FROM Bigdata.dcard_posts dp
left join Bigdata.dcard_forums df on dp.forumid = df.id
WHERE 1=1
and df.name in ('時事', '網路購物', '股票', '美妝', '工作', '考試', '穿搭', '3C', 'Apple', '感情', 
                '美食', '理財', '居家生活', '臺灣大學', 'YouTuber');
'''
df = MysqlDatabase.select_table(sql)

Successfully select from Bigdata table


In [220]:
df_t, df_v,= train_test_split(df, test_size = 0.1, random_state = 42, stratify = df.name)
df_t['type'] = 'train'
df_v['type'] = 'valid'
df = pd.concat([df_t, df_v], sort=True)
df = df.reset_index(drop=True)

/var/folders/17/dsyzw2j96ggfgmrjcsnbtz700000gn/T/ipykernel_62713/3214313076.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_t['type'] = 'train'
/var/folders/17/dsyzw2j96ggfgmrjcsnbtz700000gn/T/ipykernel_62713/3214313076.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_v['type'] = 'valid'


In [224]:
df.head(2)

,author,commentCount,createdAt,excerpt,forumId,gender,id,likeCount,name,title,topics,type
0,,49,2021-08-18 02:53:09,代po，今年表妹考指考 可是成績出來有點低於預期，國75英91數乙83歷85地86，他原本想...,d43f33f9-873f-46b3-9f24-38c90056b96c,M,236748676,16,考試,科系選擇 政大地政vs清大經濟,"['科系', '指考', '指考放榜', '填志願', '考試']",train
1,,8,2021-09-01 16:39:58,版上不知道有沒有人待過這個職缺，上次去面試他們是說這個職位就是按照計畫聘人，當沒計畫就可能被...,6eeeafb2-9dac-4d81-ae4b-ffecf0ad4444,F,236862595,2,工作,公家機關的計畫人員職缺,"['工作', '公家機關']",train


In [225]:
def regex(text):
    return text.replace("'", '').replace("[", '').replace("]", '').replace(" ", '').split(',')

In [226]:
# 看板名稱
forum_list = df.name.unique().tolist()
# 整理樣本數大於200的標籤名單
label_list = np.concatenate(df.topics.apply(regex).tolist())
df_label = pd.DataFrame.from_dict(dict(Counter(label_list)), orient='index', columns=['count']).reset_index()
df_label.columns = ['label', 'count']
df_label = df_label[(df_label['count'] >= 100) & (df_label['label'] != '')]
label_list = df_label.label.tolist()
# 合併看板名單、標籤名單
label_set = set(np.concatenate([label_list, forum_list]))


In [227]:
df.topics = df.topics.apply(regex)
df.apply(lambda x: x.topics.append(x['name']), axis=1)
df = df[['id', 'name', 'title', 'excerpt', 'topics', 'type']].drop_duplicates(subset = 'id')
df.topics = df.topics.apply(lambda x: list(set(x)))
df.topics = df.topics.apply(lambda x: set(x).intersection(label_set))
df.head()

,id,name,title,excerpt,topics,type
0,236748676,考試,科系選擇 政大地政vs清大經濟,代po，今年表妹考指考 可是成績出來有點低於預期，國75英91數乙83歷85地86，他原本想...,"{指考, 考試}",train
1,236862595,工作,公家機關的計畫人員職缺,版上不知道有沒有人待過這個職缺，上次去面試他們是說這個職位就是按照計畫聘人，當沒計畫就可能被...,{工作},train
2,234260246,美食,早餐就是要吃sukiya,「你好，現在用振興券消費，會送我們店的提袋唷！」真的嗎？好像很沒價值也，我心裡想著，什麼大風...,"{美食, 吃貨}",train
3,236806788,穿搭,Dickies台灣官網是真的嗎？,我看他粉專跟ig的粉絲都超少的,"{台灣, 穿搭}",train
4,236849649,網路購物,淘寶傢俱海運,打算9/9趁著淘寶活動 買一堆傢俱佈置房間,"{網路購物, 淘寶}",train


In [228]:
result = []
def get_long_data(postid, type, title, excerpt, topics):
    for label in topics:
        result.append([postid, type, title, excerpt, label])

In [229]:
df.apply(lambda x: get_long_data(x['id'], x.type, x.title, x.excerpt, x.topics), axis=1)

0        None
1        None
2        None
3        None
4        None
         ... 
16440    None
16442    None
16443    None
16444    None
16445    None
Length: 16128, dtype: object

In [232]:
df = pd.DataFrame(result, columns = ['id', 'type', 'title', 'excerpt', 'label'])
df['value'] = 1
df.head(2)

,id,type,title,excerpt,label,value
0,236748676,train,科系選擇 政大地政vs清大經濟,代po，今年表妹考指考 可是成績出來有點低於預期，國75英91數乙83歷85地86，他原本想...,指考,1
1,236748676,train,科系選擇 政大地政vs清大經濟,代po，今年表妹考指考 可是成績出來有點低於預期，國75英91數乙83歷85地86，他原本想...,考試,1


In [233]:
df['text'] = df['title'] + ' ' + df['excerpt']
df.head(2)

,id,type,title,excerpt,label,value,text
0,236748676,train,科系選擇 政大地政vs清大經濟,代po，今年表妹考指考 可是成績出來有點低於預期，國75英91數乙83歷85地86，他原本想...,指考,1,科系選擇 政大地政vs清大經濟 代po，今年表妹考指考 可是成績出來有點低於預期，國75英9...
1,236748676,train,科系選擇 政大地政vs清大經濟,代po，今年表妹考指考 可是成績出來有點低於預期，國75英91數乙83歷85地86，他原本想...,考試,1,科系選擇 政大地政vs清大經濟 代po，今年表妹考指考 可是成績出來有點低於預期，國75英9...


In [247]:
df1 = pd.pivot_table(df, index=['id', 'text', 'type'], columns='label', values='value', fill_value=0)

In [248]:
label_list = list(df1.columns)
label_dic = { i : label_list[i] for i in range(0, len(label_list) ) }
label_site = './model_output/dcard_cate_label_dic_2.npy'
np.save(label_site, label_dic)

In [256]:
df = df1.reset_index(drop=False)
Y_df = df1.reset_index(drop=True)

## Train Model

In [257]:
import os
import yaml
import numpy as np
import pandas as pd
from utils import mysqlDatabase
from myBertTools import myBertModel, myTokenizer
from sklearn.model_selection import train_test_split

Init Plugin
Init Graph Optimizer
Init Kernel


In [263]:
def seq_padding(X, padding=0):
    L = [len(x) for x in X]
    ML = max(L)
    return np.array([
        np.concatenate([x, [padding] * (ML - len(x))]) if len(x) < ML else x for x in X
    ])

In [264]:
class dataGenerator:
    def __init__(self, data, batch_size=32):
        self.data = data
        self.batch_size = batch_size
        self.steps = len(self.data) // self.batch_size
        if len(self.data) % self.batch_size != 0:
            self.steps += 1
    def __len__(self):
        return self.steps
    def __iter__(self):
        while True:
            idxs = list(range(len(self.data)))
            np.random.shuffle(idxs)
            X1, X2, Y = [], [], []
            for i in idxs:
                d = self.data[i]
                text = d[0][:maxlen]
                x1, x2 = MyTokenizer.encode(first=text)
                y = d[1:]
                X1.append(x1)
                X2.append(x2)
                Y.append(y)
                if len(X1) == self.batch_size or i == idxs[-1]:
                    X1 = seq_padding(X1)
                    X2 = seq_padding(X2)
                    Y = seq_padding(Y)
                    yield [X1, X2], Y
                    [X1, X2, Y] = [], [], []

In [267]:
if __name__ == '__main__':
    os.environ['TF_KERAS'] = '1'
    maxlen = 100
    pretrained_path = '/Users/jackyfu/Desktop/hwf87_git/bert_wwm/'
    config_path = os.path.join(pretrained_path, 'bert_config.json')
    checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
    vocab_path = os.path.join(pretrained_path, 'vocab.txt')
    output_path = '/Users/jackyfu/Desktop/hwf87_git/Dcard_post_classification/model_output'
    with open('config.yml', 'r') as stream:
        myconfig = yaml.load(stream, Loader=yaml.CLoader)
    database_username = myconfig['mysql_database']['database_username']
    database_password = myconfig['mysql_database']['database_password']
    database_ip       = myconfig['mysql_database']['database_ip']
    database_name     = myconfig['mysql_database']['database_name']
    # MysqlDatabase = mysqlDatabase(database_username, database_password, database_ip, database_name)
    # sql = '''
    # SELECT df.name, dp.*
    # FROM Bigdata.dcard_posts dp
    # left join Bigdata.dcard_forums df on dp.forumid = df.id
    # WHERE 1=1
    # and df.name in ('時事', '網路購物', '股票', '美妝', '工作', '考試', '穿搭', '3C', 'Apple', '感情', 
    #                 '美食', '理財', '居家生活', '臺灣大學', 'YouTuber');
    # '''
    ##
    # df, Y_df = get_data(MysqlDatabase, sql)
    MyBertModel = myBertModel(pretrained_path, config_path, checkpoint_path, vocab_path)
    token_dict = MyBertModel.get_token_dict()
    model = MyBertModel.build_model(Y_df)
    MyTokenizer = myTokenizer(token_dict)

    train_data = df[df['type'] == 'train'].drop(columns=['id', 'type']).values.tolist()
    valid_data = df[df['type'] == 'valid'].drop(columns=['id', 'type']).values.tolist()
    
    train_D = dataGenerator(train_data)
    valid_D = dataGenerator(valid_data)

    history = model.fit(
        train_D.__iter__(),
        steps_per_epoch=len(train_D),
        epochs=1,
        validation_data=valid_D.__iter__(),
        validation_steps=len(valid_D)
    )
    model_path = './model_output/dcard_post_cls_bert_2.h5'
    model.save(model_path)

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
model_10 (Functional)           (None, None, 768)    101677056   input_7[0][0]                    
                                                                 input_8[0][0]                    
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 768)          0           model_10[0][0]            

2021-09-05 19:39:33.179868: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


455/455 [==============================] - ETA: 0s - loss: 0.1085 - accuracy: 0.3040

2021-09-05 20:15:39.272510: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


455/455 [==============================] - 2279s 5s/step - loss: 0.1085 - accuracy: 0.3040 - val_loss: 0.0646 - val_accuracy: 0.5013


/Users/jackyfu/miniforge3/envs/tensorflow/lib/python3.9/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
